In [ ]:
# To run this program there are some dependencies please install these: vtk, numpy, plotly, ipywidgets
# To install vtk use below command:
# !pip install vtk

In [2]:
# To run this program on google colab execute the below code first.

# from google.colab import output
# output.enable_custom_widget_manager()

In [3]:
# Load the libraries 
import plotly.graph_objs as go
import numpy as np
from ipywidgets import Layout
import ipywidgets as widgets
import vtk

In [ ]:
# Load the VTI file
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()

# Get the image data
imageData = reader.GetOutput()

# Get the dimensions of the image data
dim = imageData.GetDimensions()

# Get the scalar data
scalarData = imageData.GetPointData().GetScalars()

# Create empty lists to store the coordinates and scalar values
x = []
y = []
z = []
values = []

# Iterate through all the points and append the coordinates and scalar values to the lists
for k in range(dim[2]):
  for j in range(dim[1]):
    for i in range(dim[0]):
      # Get the index of the point in the scalar data array
      index = imageData.ComputePointId((i, j, k))
      # print(index)

      # Get the coordinates of the point
      xCord, yCord, zCord = imageData.GetPoint(index)

      # Get the scalar value of the point
      scalarValue = scalarData.GetValue(index)

      # Append the coordinates and scalar value to the lists
      x.append(xCord)
      y.append(yCord)
      z.append(zCord)
      values.append(scalarValue)

# Convert list to numpy array
values = np.array(values)

# Define colormap and somw initial values
colormap = 'plasma'
isovalue = 0
cmin = np.min(values)
cmax = np.max(values)

# Define slider widget
slider = widgets.FloatSlider(value=isovalue, min=np.min(values), max=np.max(values), step=0.01,
                             description='Isovalue', continuous_update=False, layout=Layout(width='500px'),)

# Define reset button widget
resetBtn = widgets.Button(description='Reset', disabled=False, button_style='', tooltip='Reset',)

# We can add custom color using below
# colormap = [[0, 'rgb(13,8,135)'],
#             [0.20, 'rgb(70,3,159)'],
#             [0.40, 'rgb(114,1,168)'],
#             [0.6, 'rgb(156,23,158)'],
#             [0.7, 'rgb(189,55,134)'],
#             [0.8, 'rgb(216,87,107)'],
#             [0.9,'rgb(237,121,83)'],
#             [0.95, 'rgb(251,159,58)'],
#             [1, 'rgb(253,202,38)']]

# Create isosurface plot
trace1 = go.Isosurface(x=x, y=y, z=z, value=values, opacity=1, colorscale = colormap,
                           isomin=isovalue, isomax=isovalue, surface_count=1,showscale=False, cmin=cmin, cmax=cmax)

# Create histogram plot
trace2 = go.Histogram(x=values, opacity=0.75, name='Volume', nbinsx= 30)

# Define figure layout for isosurface
fig_layout1 = go.Layout(scene=dict(xaxis=dict(title='X',showticklabels=False),
                                   yaxis=dict(title='Y',showticklabels=False),
                                   zaxis=dict(title='Z',showticklabels=False)),
                        width=600, height=500,
                        plot_bgcolor='grey')

# Define figure layout for histogram
fig_layout2 = go.Layout(width=450, height=450, 
                        xaxis=dict(title='Vortex scalar values'), 
                        yaxis=dict(title='Frequency'))

# Create Figure Widget for isosurface and histogram
g = go.FigureWidget(data=[trace1],layout=fig_layout1)
f = go.FigureWidget(data=[trace2],layout=fig_layout2)

# Update function for slider
def response(change):
  with g.batch_update():
    g.data[0].isomax = slider.value
    g.data[0].isomin = slider.value

  with f.batch_update(): 
    a = values <= slider.value + 0.25
    b = values >= slider.value - 0.25
    newX = values[(a & b)]
    f.data[0].x = newX

slider.observe(response, names="value")

# Update function for Reset button
def resetfn(change):
  with g.batch_update():
    slider.value = 0
  with f.batch_update():
    newX = values.copy()
    f.data[0].x = newX

resetBtn.on_click(resetfn)

# Combine the widgets and figure
container1 = widgets.HBox([slider, resetBtn])
container2 = widgets.HTML(
    value="Drag the slider to change value (It will take few sec to update.)",
    description=' ',
)
container3 = widgets.HBox([g,f])

# Display all
widgets.VBox([container1,container2,container3])